In [1]:
import pandas as pd

import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

from surprise import Dataset
from surprise import Reader

from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [2]:
movies = pd.read_csv('../../ml-latest-small/movies.csv')

In [3]:
link = pd.read_csv('../../ml-latest-small/links.csv')

In [4]:
rating = pd.read_csv('../../ml-latest-small/ratings.csv')

In [5]:
tags = pd.read_csv('../../ml-latest-small/tags.csv')

In [6]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [7]:
link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [8]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [9]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


The DF tags has 'Tag" descriptions of moves provided by users. 

In [10]:
movie_rating = movies.merge(rating, on='movieId', how='outer')

In [11]:
movie_rating

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1.0,4.0,9.649827e+08
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5.0,4.0,8.474350e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7.0,4.5,1.106636e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,2.5,1.510578e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17.0,4.5,1.305696e+09
...,...,...,...,...,...,...
100849,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,184.0,4.0,1.537109e+09
100850,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,184.0,3.5,1.537110e+09
100851,193585,Flint (2017),Drama,184.0,3.5,1.537110e+09
100852,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,184.0,3.5,1.537110e+09


In [12]:
movie_rating.isna().sum()

movieId       0
title         0
genres        0
userId       18
rating       18
timestamp    18
dtype: int64

In [13]:
movie_rating.dropna(inplace=True)

In [14]:
movie_rating.shape

(100836, 6)

In [15]:
movie_rating['userId'].nunique()

610

In [16]:
stats = movie_rating[['rating', 'timestamp']].describe()
stats

,rating,timestamp
count,100836.000000,1.008360e+05
mean,3.501557,1.205946e+09
std,1.042529,2.162610e+08
min,0.500000,8.281246e+08
25%,3.000000,1.019124e+09
50%,3.500000,1.186087e+09
75%,4.000000,1.435994e+09
max,5.000000,1.537799e+09


In [17]:
print(dt.datetime.fromtimestamp(stats.loc['min', 'timestamp']))
print(dt.datetime.fromtimestamp(stats.loc['max', 'timestamp']))

1996-03-29 13:36:55
2018-09-24 09:27:30


## Split

In [1]:
from sklearn.model_selection import train_test_split

In [40]:
train, test = train_test_split(movie_rating)

In [20]:
train

,movieId,title,genres,userId,rating,timestamp
26568,1220,"Blues Brothers, The (1980)",Action|Comedy|Musical,219.0,4.5,1.194932e+09
45530,2599,Election (1999),Comedy,194.0,3.0,1.110317e+09
40642,2167,Blade (1998),Action|Horror|Thriller,249.0,4.0,1.361980e+09
68501,5796,Casino Royale (1967),Action|Adventure|Comedy,307.0,3.5,1.186088e+09
38563,2024,"Rapture, The (1991)",Drama|Mystery,202.0,3.0,9.749256e+08
...,...,...,...,...,...,...
81949,44191,V for Vendetta (2006),Action|Sci-Fi|Thriller|IMAX,318.0,3.5,1.261343e+09
74848,7373,Hellboy (2004),Action|Adventure|Fantasy|Horror,480.0,4.0,1.179165e+09
73425,7036,Teen Wolf (1985),Comedy|Fantasy,474.0,3.5,1.070287e+09
39516,2088,Popeye (1980),Adventure|Comedy|Musical,294.0,1.0,9.665968e+08


In [41]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)

^ We want to use user based.

In [45]:
model1 = KNNBasic()

In [27]:
model2 = SVD()

In [28]:
model3 = NMF()

In [29]:
model4 = knns.KNNWithMeans()

## Tuning

In [30]:
param_grid = {'k':[10, 50, 100],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.3s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.4s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.4s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   59.7s finished


In [31]:
base_model.best_params

{'rmse': {'k': 10, 'min_k': 5}, 'mae': {'k': 10, 'min_k': 5}}

In [ ]:
# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,n_jobs = -1,joblib_verbose=5)
# gs_model.fit(jokes)

## Cross Validation

In [ ]:
cv1_split = cv1.split(train_data)

In [ ]:
cv1_split

In [42]:
from surprise.model_selection import cross_validate

In [43]:
train_data

In [46]:
cross_validate(model1, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9710  0.9713  0.9563  0.9512  0.9609  0.9621  0.0080  
MAE (testset)     0.7485  0.7442  0.7342  0.7294  0.7371  0.7387  0.0069  
Fit time          0.13    0.12    0.14    0.18    0.15    0.14    0.02    
Test time         1.13    1.43    1.13    1.14    1.13    1.19    0.12    


{'test_rmse': array([0.97096217, 0.97131245, 0.95633543, 0.95117216, 0.96091782]),
 'test_mae': array([0.74849416, 0.74420727, 0.73422097, 0.72937035, 0.73711597]),
 'fit_time': (0.12800097465515137,
  0.12399649620056152,
  0.1430034637451172,
  0.17600274085998535,
  0.1490039825439453),
 'test_time': (1.1298744678497314,
  1.4259769916534424,
  1.1339972019195557,
  1.1410021781921387,
  1.1309967041015625)}

In [49]:
model1.predict(uid=10, iid  = 20)

Prediction(uid=10, iid=20, r_ui=None, est=2.7559988343808155, details={'actual_k': 7, 'was_impossible': False})